In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from datetime import timedelta
import time as datetime
from gensim.models import Word2Vec
# from feature_selector import FeatureSelector
from gensim.models import Word2Vec

from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer


from deepctr.models import xDeepFM
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [2]:
data =  pd.read_pickle('../data/train67_test.pickle')
# data = pd.read_csv('../data/test_data_A.csv',sep='|')


嵌入特征
['uid', 'task_id'] ->[adv_id，slot_id,tags]
逆向思维：

In [7]:
def w2v_id_feature(df,key1,key2,model='group',embedding_size=20,windows_size=20,workers=20,iter=10,min_count=0,use_cache=True):
    df = df[[key1,key2]]
    df[key2] =  df[key2].astype(str)
    sentence = df.groupby([key1])[key2].agg(list).reset_index()
    model_name = '../data/vector_1/{}_{}_{}.model'.format(key1,key2,embedding_size,windows_size)
    if(os.path.exists(model_name)) & (use_cache):
        model = Word2Vec.load(model_name)
    else:
        print('beging training')
        model = Word2Vec(sentence[key2].values, size=embedding_size, window = windows_size, min_count = min_count,\
                         sg=1,seed=2020,iter=10,workers=workers)
        model.save(model_name)
        print('end training')
#     df['w2v_{}_{}'.format(key1,key2)] = df.
    
    #返回均值，作为key1的代表向量
    sentence[key2] = sentence[key2].apply(lambda x: pd.DataFrame([model[c] for c in x]))
    
#     print(sentence.head())
#     print(sentence.info())
    print('beging mean compute')
    for m in range(embedding_size):
        sentence['{}_w2v_mean_{}_by_{}'.format(key1,m,key2)] = sentence[key2].apply(lambda x:x[m].mean())
        
    del sentence[key2]
    
    print('compute id_embedding')
    # 返回向量本身，作为key2的代表向量
    id_vocab = model.wv.vocab.keys() 
    id_embedding_val = [model[i] for i in id_vocab]
    id_embedding = pd.DataFrame(id_embedding_val)
    id_embedding[key2] = id_vocab
    id_embedding.columns = ['{}_w2v_{}_by_{}'.format(key2,m,key1) for m in range(embedding_size)]+[key2]
#     print(id_embedding)
    return sentence,id_embedding
        

In [4]:
# uid_mean_by_task_id, task_id_embendding_by_uid = w2v_id_feature(data.iloc[:10000],'uid','task_id')
# pd.read_feather('../data/w2vres/uid_w2v_mean_by_adv_id.feather')

In [10]:
# '
for i in tqdm(['adv_id','task_id','adv_prim_id','spread_app_id']):
    mean_by_id,id_embendding_by_uid = w2v_id_feature(data,i,'uid')
    path_name_1 = '../data/w2vres_1/uid_w2v_mean_by_{}.feather'.format(i)
    mean_by_id.to_feather(path_name_1)
    path_name_2 = '../data/w2vres_1/{}_w2v_by_uid.feather'.format(i)
    id_embendding_by_uid.to_feather(path_name_2)



  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

beging training
end training
beging mean compute
compute id_embedding




 25%|████████████████████▊                                                              | 1/4 [10:46<32:18, 646.09s/it]

beging training
end training
beging mean compute
compute id_embedding




 50%|█████████████████████████████████████████▌                                         | 2/4 [21:26<21:28, 644.44s/it]

beging training
end training
beging mean compute
compute id_embedding




 75%|██████████████████████████████████████████████████████████████▎                    | 3/4 [27:00<09:11, 551.25s/it]

beging training
end training
beging mean compute
compute id_embedding




100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [32:17<00:00, 484.43s/it]


In [14]:
# 'dev_id','slot_id','creat_type_cd','inter_typ_cd',
for i in tqdm(['tags','app_first_class','app_second_class']):
    mean_by_id,id_embendding_by_uid = w2v_id_feature(data,i,'uid')
    path_name_1 = '../data/w2vres_1/uid_w2v_mean_by_{}.feather'.format(i)
    mean_by_id.to_feather(path_name_1)
    path_name_2 = '../data/w2vres_1/{}_w2v_by_uid.feather'.format(i)
    id_embendding_by_uid.to_feather(path_name_2)




  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

beging training
end training
beging mean compute
compute id_embedding





 33%|███████████████████████████▋                                                       | 1/3 [04:56<09:53, 296.54s/it]

beging training
end training
beging mean compute
compute id_embedding





 67%|███████████████████████████████████████████████████████▎                           | 2/3 [09:43<04:53, 293.61s/it]

beging training
end training
beging mean compute
compute id_embedding





100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [14:36<00:00, 292.26s/it]


In [10]:
def gen_user_tfidf_features(df, key, column, n_components=32):
    df = df[[key, column]]
    df[column] = df[column].astype(str)
    df[column].fillna('000', inplace=True)
    df[column] = df[column].replace('-1', '000')
    group_df = df.groupby([key])[column].apply(lambda x: x.tolist()).reset_index()
    group_df.columns = [key, 'list']
    group_df['list'] = group_df['list'].apply(lambda x: 'n' + ',n'.join(x))
    enc_vec = TfidfVectorizer()
    tfidf_vec = enc_vec.fit_transform(group_df['list'])
    # print(enc_vec.get_feature_names())
    
    nunq = df[column].nunique()
    if nunq < n_components:
        n_components = nunq
        vec_svd = pd.DataFrame(tfidf_vec.toarray())
        # print(df[column].unique(), vec_svd.shape)
        vec_svd.columns = ['{}_tfidf_{}_{}'.format(key,column, i) for i in range(n_components)]
        group_df = pd.concat([group_df, vec_svd], axis=1)
    else: 
        svd_enc = TruncatedSVD(n_components=n_components, n_iter=10, random_state=2020)
        vec_svd = svd_enc.fit_transform(tfidf_vec)
        vec_svd = pd.DataFrame(vec_svd)
        vec_svd.columns = ['{}_svd_tfidf_{}_{}'.format(key,column, i) for i in range(n_components)]
        group_df = pd.concat([group_df, vec_svd], axis=1)
    del group_df['list']
    file_path = '../data/tfidfres/'+key+'_'+'tfidfsvd'+'_by_'+column+'.feather'
    group_df.to_feather(file_path)
#     return group_df

In [4]:
cols2 = ['adv_prim_id', 'tags', 'app_first_class']
for col in tqdm(cols2): 
    gen_user_tfidf_features(data[['uid',col]],'uid',col)

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

begining svd


 33%|███████████████████████████▋                                                       | 1/3 [01:40<03:20, 100.42s/it]

begining svd


 67%|████████████████████████████████████████████████████████                            | 2/3 [03:02<01:34, 94.88s/it]

begining svd


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [04:23<00:00, 87.91s/it]


In [11]:
# 'tags', 'app_first_class', 'career', 
cols2 =  ['age', 'gender', 'device_price', 'creat_type_cd',  'slot_id', 'city_rank']
for col in tqdm(cols2): 
    gen_user_tfidf_features(data[['adv_prim_id',col]],'adv_prim_id',col)




  0%|                                                                                            | 0/6 [00:00<?, ?it/s]


 17%|██████████████                                                                      | 1/6 [00:24<02:03, 24.68s/it]


 33%|████████████████████████████                                                        | 2/6 [00:40<01:28, 22.08s/it]


 50%|██████████████████████████████████████████                                          | 3/6 [00:57<01:01, 20.37s/it]


 67%|████████████████████████████████████████████████████████                            | 4/6 [01:13<00:38, 19.06s/it]


 83%|██████████████████████████████████████████████████████████████████████              | 5/6 [01:29<00:18, 18.28s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:46<00:00, 17.71s/it]


In [12]:
cols2 =  ['adv_prim_id', 'age', 'gender', 'career', 'slot_id', 'city_rank']
for col in tqdm(cols2): 
    gen_user_tfidf_features(data[['spread_app_id',col]],'spread_app_id',col)




  0%|                                                                                            | 0/6 [00:00<?, ?it/s]


 17%|██████████████                                                                      | 1/6 [00:16<01:20, 16.13s/it]


 33%|████████████████████████████                                                        | 2/6 [00:32<01:04, 16.08s/it]


 50%|██████████████████████████████████████████                                          | 3/6 [00:47<00:47, 15.95s/it]


 67%|████████████████████████████████████████████████████████                            | 4/6 [01:03<00:31, 15.87s/it]


 83%|██████████████████████████████████████████████████████████████████████              | 5/6 [01:19<00:15, 15.89s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:35<00:00, 15.86s/it]


In [2]:
# for i in tqdm(['adv_id','task_id','adv_prim_id','spread_app_id','dev_id','slot_id']):
#     tmp = tfidf(data[['uid',i]],'uid',i,20,'uid_'+i)
#     path_name_1 = '../data/tfidfres/uid_tfidf_by_{}.feather'.format(i)
#     tmp.to_feather(path_name_1)

# Target Encoder


In [4]:
train = data[~data['label'].isna()]
test = data[data['label'].isna()]

In [8]:
#  对ID特征进行 Target Encoder
def kfold_stats_feature(train,test,feats,k):
    folds = StratifiedKFold(n_splits=k,shuffle=True,random_state=2020)
    train['fold'] = None
    for fold_,(trn_idx, val_idx) in enumerate(folds.split(train,train['label'])):
        train.loc[val_idx,'fold'] = fold_
        
    kfold_features = []
    for feat in tqdm(feats):
        nums_columns = ['label']
        for f in nums_columns:
            colname = feat + '_' + f + '_kfold_mean'
            kfold_features.append(colname)
            train[colname] = None
            for fold_,(trn_idx,val_idx) in enumerate(folds.split(train,train['label'])):
                order_label = train.iloc[trn_idx].groupby([feat])[f].mean()
                train.loc[train.fold == fold_, colname] = train.loc[train.fold == fold_, [feat]][feat].map(order_label)
                
                # fillna
                global_mean = train[f].mean()
                train.loc[train.fold == fold_, colname] = train.loc[train.fold == fold_, colname].fillna(global_mean)
            train[colname] = train[colname].astype(float)
            
            
        for f in nums_columns:
            colname = feat + '_' + f + '_kfold_mean'
            test[colname] = None
            order_label = train.groupby([feat])[f].mean()
            test[colname] = test[feat].map(order_label)
            test[colname] = test[colname].astype(float)        
    del train['fold']
    return train,test

In [9]:
target_encode_cols = ['task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id', 
                  	  'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags', 'app_first_class',]
train,test =kfold_stats_feature(train,test,target_encode_cols,5)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:03<00:00, 12.31s/it]


（1）组合点击率特征
（2）slot_id的分布，
（3）下一次点击网络变化
df_feature['next_netmodel'] = df_feature.groupby(['deviceid'])[ 'netmodel'].shift(-1)
（4） 对uid 进行分组，得到每个task_id对应的多个u_id，word2vec,得到每个广告的vec表示形式
（5) 每个gender age  career  city city_rank device_name点击的广告行业的均值

In [3]:
#  对ID特征进行 Target Encoder
def mix_kfold_stats_feature(train,test,mix_1,mix_2,k):
    folds = StratifiedKFold(n_splits=k,shuffle=True,random_state=2020)
    train['fold'] = None
    for fold_,(trn_idx, val_idx) in enumerate(folds.split(train,train['label'])):
        train.loc[val_idx,'fold'] = fold_
        
    kfold_features = []
    for feat_1 in tqdm(mix_1):
        for feat_2 in mix_2:
            nums_columns = ['label']
            for f in nums_columns:
                mix1_mix_2_col = feat_1 +'_'+ feat_2
                colname = feat_1 +'_'+ feat_2 + '_' + f + '_kfold_mean'
                kfold_features.append(colname)
                train[colname] = None
                train[mix1_mix_2_col] =  train[feat_1].astype(str) + '_'+ train[feat_2].astype(str)
                test[mix1_mix_2_col] =  test[feat_1].astype(str) + '_'+ test[feat_2].astype(str)

                for fold_,(trn_idx,val_idx) in enumerate(folds.split(train,train['label'])):
                    order_label = train.iloc[trn_idx].groupby([mix1_mix_2_col])[f].mean()
#                     print(order_label.reset_index())
                    train.loc[train.fold == fold_, colname] =\
                            train.loc[train.fold == fold_, [mix1_mix_2_col]][mix1_mix_2_col].map(order_label)

#                     # fillna
                    global_mean = train[f].mean()
                    train.loc[train.fold == fold_, colname] = train.loc[train.fold == fold_, colname].fillna(global_mean)
                train[colname] = train[colname].astype(float)


            for f in nums_columns:
                colname = feat_1 +'_'+ feat_2 + '_' + f + '_kfold_mean'
                test[colname] = None
                order_label = train.groupby([mix1_mix_2_col])[f].mean()
                test[colname] = test[mix1_mix_2_col].map(order_label)
                test[colname] = test[colname].astype(float)
            del train[mix1_mix_2_col]
            del test[mix1_mix_2_col]
    del train['fold']
    return train,test

In [11]:
# 组合交叉点击率
# task_id(广告id) / ndu_name(广告行业) /adv_prim_id(广告主) 与gender age  career  city city_rank device_name
mix_1_traget_encode_cols = ['task_id','adv_id','indu_name','adv_prim_id','spread_app_id','tags','app_first_class','app_second_class']
mix_2_traget_encode_cols = ['age','gender','career','city','city_rank','device_name','device_size','residence','his_app_size']
train,test = mix_kfold_stats_feature(train,test,mix_1_traget_encode_cols,mix_2_traget_encode_cols,5)


100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [29:00<00:00, 217.59s/it]


## # 三阶交叉


In [19]:
#  对ID特征进行 Target Encoder
def mix_3_kfold_stats_feature(train,test,mix_1,mix_2,k):
    folds = StratifiedKFold(n_splits=k,shuffle=True,random_state=2020)
    train['fold'] = None
    for fold_,(trn_idx, val_idx) in enumerate(folds.split(train,train['label'])):
        train.loc[val_idx,'fold'] = fold_
        
    kfold_features = []
    for feat_1 in tqdm(mix_1):
        for i,feat_2 in enumerate(mix_2[:-1]):
            
            mix_3 = mix_2[i+1:]
            for feat_3 in mix_3:
                nums_columns = ['label']
                for f in nums_columns:
                    mix2_mix_3_col =  feat_1 +'_'+feat_2 +'_'+ feat_3
                    colname = feat_1 +'_'+ feat_2 + feat_3 + '_' + f + '_kfold_mean'
                    kfold_features.append(colname)
                    train[colname] = None
                    train[mix2_mix_3_col] = train[feat_1].astype(str) + '_'+ train[feat_2].astype(str) + '_'+ train[feat_3].astype(str)
                    test[mix2_mix_3_col] =  test[feat_1].astype(str) + '_'+ test[feat_2].astype(str) + '_'+ test[feat_3].astype(str)

                    for fold_,(trn_idx,val_idx) in enumerate(folds.split(train,train['label'])):
                        order_label = train[[mix2_mix_3_col,f]].iloc[trn_idx].groupby([mix2_mix_3_col])[f].mean()
    #                     print(order_label.reset_index())
                        train.loc[train.fold == fold_, colname] =\
                                train.loc[train.fold == fold_, [mix2_mix_3_col]][mix2_mix_3_col].map(order_label)

    #                     # fillna
                        global_mean = train[f].mean()
                        train.loc[train.fold == fold_, colname] = train.loc[train.fold == fold_, colname].fillna(global_mean)
                    train[colname] = train[colname].astype(float)


                for f in nums_columns:
                    colname = feat_1 +'_'+ feat_2 + feat_3 + '_' + f + '_kfold_mean'
                    test[colname] = None
                    order_label = train[[mix2_mix_3_col,f]].groupby([mix2_mix_3_col])[f].mean()
                    test[colname] = test[mix2_mix_3_col].map(order_label)
                    test[colname] = test[colname].astype(float)
                del train[mix2_mix_3_col]
                del test[mix2_mix_3_col]
    del train['fold']
    return train,test

In [20]:
# 组合交叉点击率,用户画像交叉与广告属性的labelEncoding
# task_id(广告id) / ndu_name(广告行业) /adv_prim_id(广告主) 与gender age  career  city city_rank device_name
# ,'adv_id','indu_name','adv_prim_id','spread_app_id','tags','app_first_class','app_second_class'
mix_1_traget_encode_cols = ['task_id','indu_name','adv_prim_id']
mix_2_traget_encode_cols = ['age','gender','career','city_rank','residence']
# 'city','device_name','device_size',,'his_app_size'
train,test = mix_3_kfold_stats_feature(train,test,mix_1_traget_encode_cols,mix_2_traget_encode_cols,5)


100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [21:36<00:00, 432.22s/it]


In [32]:
train.shape

Index(['task_id_agegender_label_kfold_mean',
       'task_id_agecareer_label_kfold_mean',
       'task_id_agecity_rank_label_kfold_mean',
       'task_id_ageresidence_label_kfold_mean',
       'task_id_gendercareer_label_kfold_mean',
       'task_id_gendercity_rank_label_kfold_mean',
       'task_id_genderresidence_label_kfold_mean',
       'task_id_careercity_rank_label_kfold_mean',
       'task_id_careerresidence_label_kfold_mean',
       'task_id_city_rankresidence_label_kfold_mean',
       'indu_name_agegender_label_kfold_mean',
       'indu_name_agecareer_label_kfold_mean',
       'indu_name_agecity_rank_label_kfold_mean',
       'indu_name_ageresidence_label_kfold_mean',
       'indu_name_gendercareer_label_kfold_mean',
       'indu_name_gendercity_rank_label_kfold_mean',
       'indu_name_genderresidence_label_kfold_mean',
       'indu_name_careercity_rank_label_kfold_mean',
       'indu_name_careerresidence_label_kfold_mean',
       'indu_name_city_rankresidence_label_kfold_mea

In [37]:

data = pd.concat([train,test],sort=False).reset_index(drop=True)

In [38]:
data = data_compression(data)

  0%|                                                                                           | 0/66 [00:00<?, ?it/s]

Memory usage of dataframe is 6559.70 MB


100%|██████████████████████████████████████████████████████████████████████████████████| 66/66 [00:56<00:00,  1.18it/s]

Memory usage after optimization is: 1317.00 MB
Decreased by 79.9%


In [40]:
data.to_feather('../data/763/add_3_fold.feather')

In [41]:
data = data[data.columns[36:]]

In [43]:
data = data.fillna(0)

In [47]:
data.head()

""
0
1
2
3
4


In [46]:
import gc
tmp_feat = pd.DataFrame()
for feat in tqdm(data.columns):
    tmp_feat[feat] = MinMaxScaler(feature_range=(0,1)).fit_transform(data[[feat]]).astype(np.float16)[:,0]
    del data[feat]
    gc.collect()

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:16<00:00,  1.80it/s]


In [49]:
tmp_feat.to_feather('../data/763/add_3_fold_scale.feather')

#  Count Encoder

In [13]:
# Count Encoder
to_count = [['task_id'], ['adv_id'], ['creat_type_cd'], ['adv_prim_id'], 
            ['dev_id'], ['inter_type_cd'], ['slot_id'], ['spread_app_id'], ['tags'], ['app_first_class'],
            ['app_second_class'], ['age'], ['city'], ['city_rank'], ['device_name'], ['device_size'],
            ['career'], ['gender'], ['net_type'], ['residence'], ['his_app_size'], ['his_on_shelf_time'],
            ['app_score'], ['emui_dev'], ['list_time'], ['device_price'], ['up_life_duration'], ['up_membership_grade'],
            ['membership_life_duration'], ['consume_purchase'], ['communication_onlinerate'], ['communication_avgonline_30d'],
            ['indu_name']]

for i in tqdm(to_count):
    data['{}_count'.format('_'.join(i))] = data[i].groupby(i)[i].transform('count')
    

100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [01:09<00:00,  2.11s/it]


In [14]:
# data.isna().sum()

In [15]:
# task_id分组（交叉） Count Encoder
to_count_1 = ['adv_id','creat_type_cd','adv_prim_id','dev_id','spread_app_id','tags','app_first_class','app_second_class','indu_name','inter_type_cd']
for col in to_count_1:
    data["{}_count".format("_".join(i))] = data[['task_id',col]].groupby(['task_id'])[col].transform('count')


In [16]:
# u_id 分组（交叉） Count Encoder
# 同一个广告推荐给用户的次数
data['uid_adv_id_cnt'] = data[['uid','adv_id']].groupby(['uid','adv_id'])['adv_id'].transform('count')

In [17]:
# 每个广告在不同地区，age的推荐次数
mix_1_count_cols = ['task_id','adv_id','indu_name','adv_prim_id','spread_app_id','tags','app_first_class','app_second_class']
mix_2_count_cols = ['age','gender','career','city','city_rank','device_name','device_size','residence','his_app_size']
for i in mix_1_count_cols:
    for j in mix_2_count_cols:
         data[i+'_'+j+'_'+'cnt'] = data[[i,j]].groupby([i,j])[j].transform('count')

# Nunique Encoder

In [18]:
# Count Encoder
to_group = [
    ['uid','task_id'], ['uid','adv_id'], ['uid','adv_prim_id'], ['uid','dev_id'], ['uid','slot_id'],
    ['uid','spread_app_id'], ['uid','app_first_class'], ['uid','city'], ['uid','device_name'], ['uid', 'net_type'],
    ['uid','communication_onlinerate'], ['uid','list_time']
]

feature = pd.DataFrame()
for i in tqdm(to_group):
    data["STAT_{}_nunique_1".format("_".join(i))] = data[i].groupby(i[1])[i[0]].transform('nunique')
    data["STAT_{}_nunique_2".format("_".join(i))] = data[i].groupby(i[0])[i[1]].transform('nunique')
    data["COUNT-2order_{}".format("_".join(i))] = data[i].groupby(i)[i[0]].transform("count")

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [09:46<00:00, 48.91s/it]


In [23]:
data.columns

Index(['label', 'uid', 'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
       'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id',
       ...
       'COUNT-2order_uid_device_name', 'STAT_uid_net_type_nunique_1',
       'STAT_uid_net_type_nunique_2', 'COUNT-2order_uid_net_type',
       'STAT_uid_communication_onlinerate_nunique_1',
       'STAT_uid_communication_onlinerate_nunique_2',
       'COUNT-2order_uid_communication_onlinerate',
       'STAT_uid_list_time_nunique_1', 'STAT_uid_list_time_nunique_2',
       'COUNT-2order_uid_list_time'],
      dtype='object', length=260)

In [23]:
# 内存压缩
def data_compression(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in tqdm(df.columns):
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

74

# 统计特征

In [7]:
to_group = [['task_id'], ['dev_id'], ['adv_prim_id'], ['adv_id'], ['inter_type_cd'], ['slot_id'], ['tags'], ['app_first_class']]
to_inter =  [ 'age', 'city_rank', 'career', 'his_app_size', 'his_on_shelf_time', 'app_score', 'emui_dev', 'device_price', 'up_life_duration', 'communication_avgonline_30d']
# to_calc = ['std','mean','min','max',lambda x:np.std(np.fft.fft(x))]
to_calc = ['std','mean','min','max']

for i in tqdm(to_group):
    for j in to_inter:
        for k in to_calc:
            data['STAT_{}_{}_{}'.format('_'.join(i),j,k)] = data[i + [j]].groupby(i)[j].transform(k)



100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [58:58<00:00, 442.29s/it]


In [6]:
# 压缩
data = data_compression(data)

Memory usage of dataframe is 13993.34 MB
Memory usage after optimization is: 7044.81 MB
Decreased by 49.7%


In [7]:
import gc
gc.collect()

27

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6302870 entries, 0 to 6302869
Columns: 580 entries, label to STAT_app_first_class_communication_avgonline_30d_max
dtypes: category(1), float16(241), int16(20), int32(110), int8(208)
memory usage: 6.9 GB


In [25]:
# 特征选择
drop_feat = ['label','id','uid','pt_d']
feature_name = [i for i in data.columns if i not in drop_feat]
useless_col = []
for col in tqdm(feature_name):
    if (data[col].value_counts().iloc[0] / data_len) > 0.95:
        useless_col.append(col)


 32%|█████████████████████████▎                                                      | 183/577 [00:22<00:42,  9.20it/s]


 77%|█████████████████████████████████████████████████████████████▋                  | 445/577 [00:45<00:11, 11.63it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 577/577 [00:57<00:00, 10.11it/s]


In [31]:
useless_col

['membership_life_duration',
 'membership_life_duration_count',
 'STAT_uid_city_nunique_2',
 'STAT_uid_device_name_nunique_2',
 'STAT_uid_list_time_nunique_2',
 'STAT_task_id_age_max',
 'STAT_task_id_city_rank_min',
 'STAT_task_id_city_rank_max',
 'STAT_task_id_career_min',
 'STAT_task_id_his_app_size_std',
 'STAT_task_id_his_on_shelf_time_std',
 'STAT_task_id_app_score_std',
 'STAT_task_id_up_life_duration_min',
 'STAT_task_id_up_life_duration_max',
 'STAT_task_id_communication_avgonline_30d_min',
 'STAT_task_id_communication_avgonline_30d_max',
 'STAT_dev_id_age_min',
 'STAT_dev_id_age_max',
 'STAT_dev_id_city_rank_min',
 'STAT_dev_id_city_rank_max',
 'STAT_dev_id_career_min',
 'STAT_dev_id_career_max',
 'STAT_dev_id_his_on_shelf_time_std',
 'STAT_dev_id_app_score_std',
 'STAT_dev_id_emui_dev_min',
 'STAT_dev_id_emui_dev_max',
 'STAT_dev_id_device_price_min',
 'STAT_dev_id_device_price_max',
 'STAT_dev_id_up_life_duration_min',
 'STAT_dev_id_up_life_duration_max',
 'STAT_dev_id_commu

In [3]:
# 回调 随机游走


In [33]:
data.drop(columns=useless_col,inplace=True)

In [35]:
data.to_feather('../data/data_feat_all_reduce.feather')

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6302870 entries, 0 to 6302869
Columns: 455 entries, label to STAT_app_first_class_communication_avgonline_30d_mean
dtypes: category(1), float16(233), int16(20), int32(109), int8(92)
memory usage: 6.1 GB


# train and test

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from datetime import timedelta
import time as datetime
from gensim.models import Word2Vec
# from feature_selector import FeatureSelector
from gensim.models import Word2Vec

from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer


from deepctr.models import xDeepFM
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [ ]:
i_sparse_features = ['uid','task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
                   'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags',
                   'app_first_class', 'app_second_class','city', 'city_rank',
                   'device_name','career', 'gender', 'net_type', 'residence', 
                   'consume_purchase','indu_name','label']
%time sparse_data = pd.read_feather('../data/data_feat_all_reduce.feather',columns = i_sparse_features)
%time dense_feature_scaler = pd.read_feather('../data/dense_feature_scaler.feather')
pt_d  = pd.read_feather('../data/data_feat_all_reduce.feather',columns = ['pt_d'])



Wall time: 935 ms


In [3]:
sparse_features = ['task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
                   'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags',
                   'app_first_class', 'app_second_class','city', 'city_rank',
                   'device_name','career', 'gender', 'net_type', 'residence', 
                   'consume_purchase','indu_name',]
sparse_data[sparse_features] = sparse_data[sparse_features].fillna('-1', )
types_map = sparse_data.dtypes
for feat in sparse_features:
    sparse_data[feat] = LabelEncoder().fit_transform(sparse_data[feat]).astype(types_map[feat])

In [4]:
data = pd.concat([sparse_data,dense_feature_scaler],axis=1)
data = pd.concat([data,pt_d],axis=1)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6302870 entries, 0 to 6302869
Columns: 454 entries, uid to pt_d
dtypes: float16(432), int16(4), int32(1), int8(17)
memory usage: 5.2 GB


In [6]:
drop_feat = ['label','id','uid','pt_d','communication_onlinerate']
feature_name = [i for i in data.columns if i not in drop_feat]

In [7]:

# membership_life_duration communication_onlinerate
sparse_features = ['task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
                   'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags',
                   'app_first_class', 'app_second_class','city', 'city_rank',
                   'device_name','career', 'gender', 'net_type', 'residence', 
                   'consume_purchase','indu_name',]
dense_features_1 = ['age', 'device_size',  'his_app_size', 'his_on_shelf_time', 'app_score',
                   'emui_dev', 'list_time', 'device_price', 'up_life_duration',
                  'communication_avgonline_30d']
dense_features_2 = [i for i in feature_name if i not in (sparse_features+dense_features_1)]
dense_features = dense_features_2 + dense_features_1
target = ['label']


In [8]:
print('begin fillna....')
data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0,)

types_map = data.dtypes
for feat in sparse_features:
    data[feat] = LabelEncoder().fit_transform(data[feat]).astype(types_map[feat])
    
print('begin minmaxScaler')  
for feat in tqdm(dense_features):
    data[[feat]] = MinMaxScaler(feature_range=(0,1)).fit_transform(data[[feat]]).astype(np.float16)

In [9]:
fixlen_feature_columns  = [SparseFeat(feat,vocabulary_size=data[feat].nunique(),embedding_dim=4) for feat in sparse_features]\
            + [DenseFeat(feat,1,) for feat in dense_features]

In [3]:
# 生成特征列
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

NameError: name 'fixlen_feature_columns' is not defined

In [11]:
# train = data[data['pt_d'] == 6].reset_index(drop=True)
# val = data[data['pt_d'] == 7].reset_index(drop=True)

test = data[data['pt_d'] == 8].reset_index(drop=True)

In [12]:
del data 
import gc
gc.collect()

47

In [13]:
train_model_input = {name: train[name].values for name in feature_names}
val_model_input = {name: val[name].values for name in feature_names}
test_model_input = {name: test[name].values for name in feature_names}

In [15]:
model = xDeepFM(linear_feature_columns,dnn_feature_columns,task='binary')
model.compile('adam','binary_crossentropy',metrics=['binary_crossentropy','accuracy'])

history = model.fit(train_model_input,train[target].values,batch_size=4096,epochs=2,
                    validation_data = (val_model_input,val[target].values))


Train on 2630371 samples, validate on 2672499 samples
Epoch 1/2
2630371/2630371 [==============================] - 103s 39us/sample - loss: 0.2283 - binary_crossentropy: 0.2282 - accuracy: 0.9240 - val_loss: 0.2161 - val_binary_crossentropy: 0.2160 - val_accuracy: 0.9282
Epoch 2/2
2630371/2630371 [==============================] - 92s 35us/sample - loss: 0.2147 - binary_crossentropy: 0.2146 - accuracy: 0.9286 - val_loss: 0.2191 - val_binary_crossentropy: 0.2190 - val_accuracy: 0.9277


In [17]:
pred_ans = model.predict(val_model_input,batch_size=4096*2)
roc_auc_score(val[target].values,pred_ans)

0.8117351026636368

In [2]:
# 第二次训练，验证集作为训练

history = model.fit(val_model_input,val[target].values,batch_size=4096,epochs=2,)

NameError: name 'linear_feature_columns' is not defined

In [19]:
pred_ans = model.predict(val_model_input,batch_size=4096*2)
roc_auc_score(val[target].values,pred_ans)

0.818748929995436

In [20]:
pred_ans = model.predict(test_model_input,batch_size=4096*4)

In [23]:
submit = pd.DataFrame()
submit['id'] = range(1,len(test)+1)
submit['probability'] = pred_ans


In [24]:
submit.head()

,id,probability
0,1,0.017800
1,2,0.356191
2,3,0.505414
3,4,0.040682
4,5,0.123632


In [25]:
submit.to_csv('../submit/0906_1/submission.csv',index=False)

In [17]:
test_pred_prob

array([0.01327293, 0.15979464, 0.23355472, ..., 0.15256163, 0.08342812,
       0.07438161])

In [18]:
submit = pd.DataFrame()
submit['id'] = range(1,len(test)+1)
submit['probability'] = test_pred_prob


In [20]:
submit.to_csv('../submit/0903_2/submission.csv',index=False)

In [21]:
prob_oof

array([0.01163306, 0.06300333, 0.02498697, ..., 0.01603606, 0.01725756,
       0.02520344])

In [18]:
t1 = pd.DataFrame({'id':[1,2,3],'to':[2,3,4],'label':[9,6,7]})
t1

,id,to
0,1,2
1,2,3
2,3,4


In [19]:
t2 = pd.DataFrame({'ids':[1,2,3],'tos':[2,3,4]})
t2

,ids,tos
0,1,2
1,2,3
2,3,4


In [20]:
t2[['ids','tos']].map(t1)

AttributeError: 'DataFrame' object has no attribute 'map'